In [ ]:
## ***Uncertainty Quantification***

Exercise 3

Prof. Dr. Martin Frank

#### **EXERCISE SHEET 3**

In [ ]:
!pip install chaospy
import numpy as np
import math
import scipy.special
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt
from matplotlib import cm
import chaospy.distributions.copulas.clayton

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   

**EXERCISE 1)**

Consider the ODE with uncertainties
$$
\dot u(t,z) = -z_1 u(t,z),\quad u(0,z)=z_2.
$$ 
Derive expressions for the
sensitivities of 
$$R(u(z)) = \int_0^T u(t,z)dt$$ 
with respect to $z_1$ and $z_2$ by

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (a) using the explicit solution,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (b) using forward differentiation,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (c) using adjoints.


&nbsp;

**EXERCISE 2)**

Assume a (discretized) system can be written in the form
$$ E(u,z) = 0,$$

where
- the solution $u$ $\in \mathbb{R}^K$
- the parameters $z$ $\in \mathbb{R}^N$

and the constraint E: $\mathbb{R}^K \times \mathbb{R}^N \rightarrow \mathbb{R}^K$ admits a unique solution for $u$ depending on $z$. Assume further that we have quantities of interest
$$R(u) = R(u(z))$$
with $R: \mathbb{R}^K \rightarrow \mathbb{R}^I$. We are interested in the sensitivity of $R$ with respect to $z$, i.e.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (a) Derive formulas for the sensitivities by forward-differentiation.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (b) Derive formulas for the sensitivities by adjoint calculus.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (c) Discuss the dimensions of all objects. When is forward differentiation advantageous, when adjoints?

&nbsp;


In [ ]:

&nbsp;

### **Additional Exercises**

&nbsp;

**EXERCISE 3)**

We want to compute the expected value of $g(Z)=\sqrt{Z} sin(2\pi Z)$, where $Z$ is uniformly distributed in [0,1], with the help of Monte-Carlo. We use control variates to speed up the computation. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (a) Plot $g$ and the control variate $\lambda$ by running the following block of code. Choose different values of $\lambda$ and try to get 



In [ ]:
#Define g and h

def g(Z):
  y=math.sqrt(Z)*math.sin(2*math.pi*Z)
  return y

def h(Z):
  y=math.sin(2*math.pi*Z)
  return y 

g=np.vectorize(g)
h=np.vectorize(h)

#Plot g and control variate h
lambda_c = 10
zGrid = np.linspace(start=0,stop=1,num=1000)
plt.figure(figsize=(8,5))
plt.plot(zGrid,g(zGrid),label='g')
plt.plot(zGrid,lambda_c*h(zGrid),linestyle='--',linewidth=2,label='h')
_=plt.legend(shadow=True, fancybox=True)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (b) Compute the expected value of $g$ by using Monte Carlo with and without control variate. How does the choice of $\lambda$ affect the solution?

In [ ]:
N=5000  #number of samples

tmpMC = 0
tmpCV = 0

expectedMC = np.zeros((N,1))
expectedCV = np.zeros((N,1))

for i in range(N):
  Z=np.random.uniform(0,1,(1,1))

  #Your code: Save MC and CV expectation value approximation with i+1 samples in expectedMC(i) and expectedCV(i)
  #...
  #expectedMC[i], expectedCV[i], tmpMC, tmpCV = solution_b(tmpMC,tmpCV,Z,lambda_c,i+1) #(run code section below exercise c) & uncomment this line to see solution)

print("Expected value is {0}".format(expectedMC[N-1]))

expectedExact = -0.11980826271647446

plt.figure(figsize=(8,5))
plt.loglog(np.arange(1,N+1),abs(expectedMC-expectedExact),label='MC')
plt.loglog(np.arange(1,N+1),abs(expectedCV-expectedExact),label='CV')
plt.loglog(np.arange(1,N+1),1/np.sqrt(np.arange(1,N+1)),linestyle='--',linewidth=2,label='slope 1/2')
_=plt.legend(shadow=True, fancybox=True)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (c) Look at the code below. What is implemented here? How is the value of $\lambda$ picked?

In [ ]:
def estimateCov (covOld,QgOld,N,Z):
  y=(N-1)/N*covOld + 1/N* (g(Z)-QgOld)* (h(Z)-QgOld)
  return y

#Compute integral with CV and optimal lambda

expectedCVL = np.zeros((N,1))
tmpCVL = 0
covOld = 0.5           #estimated covariance
expectedOld = -0.1     #estimated mean
varH = 0.5             #exact variance of function h

for i in range(N):
  Z = np.random.uniform(0,1,(1,1))
  cov = estimateCov(covOld, expectedOld, i+1 , Z) #update covariance
  lambdaOpt = cov/varH
  tmpCVL = tmpCVL + (g(Z) - lambdaOpt*h(Z))
  expectedCVL[i] = 1/(i+1)*tmpCVL
  covOld = cov
  expectedOld = expectedCVL[i]

print("Expected value is {0}".format(expectedCVL[N-1]))

expectedExact = -0.11980826271647446
plt.figure(figsize=(8,5))
plt.loglog(np.arange(1,N+1),abs(expectedMC-expectedExact),label='MC')
plt.loglog(np.arange(1,N+1),abs(expectedCV-expectedExact),linestyle='--',linewidth=2,label='CV')
plt.loglog(np.arange(1,N+1),abs(expectedCVL-expectedExact),linestyle='-.',linewidth=2,label='CVOpt')
plt.loglog(np.arange(1,N+1),1/np.sqrt(np.arange(1,N+1)),label='slope 1/2')
_=plt.legend(shadow=True, fancybox=True)

print("Lambda is {0}".format(lambdaOpt))

plt.figure(figsize=(8,5))
plt.plot(zGrid,g(zGrid),label='g')
plt.plot(zGrid,np.squeeze(lambdaOpt*h(zGrid)),label='$\lambda_{Opt} \; h$')
plt.plot(zGrid,lambda_c*h(zGrid),linestyle='--',linewidth=2,label='$\lambda \; h$')
_=plt.legend(shadow=True, fancybox=True)

In [ ]:
#Solution to fill in field in (b)

def solution_b (tmpMC,tmpCV,Z,lambda_c,i):
  tmpMC = tmpMC + g(Z);
  tmpCV = tmpCV + (g(Z)-lambda_c*h(Z));
  expectedMC_i = (1.0/i)*tmpMC;
  expectedCV_i = (1.0/i)*tmpCV;

  return expectedMC_i, expectedCV_i, tmpMC, tmpCV